In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U spacy
!pip install -U scispacy
!pip install -U beautifulsoup4
!pip install -U periodictable
!pip install -U pandas
!pip install -U numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13578641 sha256=01fec4a7c9c0448528aa4a10d3931dabe83105b7ea0fd08a8f4678cb685610a4
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Crea

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import spacy
import scispacy
import numpy as np

In [5]:
from spacy.tokens.doc import Doc
from spacy.tokens.token import Token
from spacy.tokens.span import Span
from pandas.core.frame import DataFrame
from numpy import ndarray

In [6]:
GENERAL_PATH = "/content/drive/MyDrive/gena-db-master/src/DATASET"
DATA_PATH = f"{GENERAL_PATH}/data"
RESULT_PATH = f"{GENERAL_PATH}/results"

In [7]:
import spacy

from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/gena-db-master/src/DATASET/results/model/en_gena_sm'

gena_nlp = spacy.load(model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.0) was trained with spaCy v3.2.3 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# Normalize the entities in result file

In [8]:
results_df = pd.read_csv(f"{RESULT_PATH}/raw_result/gena_data_raw_3.csv")
results_df.head(5)

,PMID,Sentence,E1,Type_E1,Relation,E2,Type_E2
0,31792039,QUESTION 1: What nutritional/metabolic test(s)...,metabolic disorders,DISEASE,associated with,low carnitine levels?Organic acidaemiasFatty a...,DISEASE
1,31792039,QUESTION 1: What nutritional/metabolic test(s)...,metabolic disorders,DISEASE,associated with,low carnitine levels?Organic acidaemiasFatty a...,CHEMICAL
2,31019473,This is more evident during rapid growth (i.e....,thiamine deficiency,NUTRITION,associated in,malnutrition,DISEASE
3,31019473,Thiamine deficiency contributes to a number of...,confusion,DISEASE,reduce memory to,ataxia,DISEASE
4,31019473,Thiamine deficiency contributes to a number of...,confusion,DISEASE,reduce memory to,death,DISEASE


In [9]:
import pandas as pd
papers_df = pd.read_csv(f"{DATA_PATH}/papers_raw/final_papers_3.csv")
full_df = pd.merge(results_df, papers_df, how='left', left_on=['PMID'], right_on=['PMID'])
full_df = full_df[['PMID', 'Sentence', 'Abstract', 'E1', 'Type_E1', 'Relation', 'E2', 'Type_E2']]
full_df.head(50)

,PMID,Sentence,Abstract,E1,Type_E1,Relation,E2,Type_E2
0,31792039,QUESTION 1: What nutritional/metabolic test(s)...,A 9-year-old boy presented with a 2-day histor...,metabolic disorders,DISEASE,associated with,low carnitine levels?Organic acidaemiasFatty a...,DISEASE
1,31792039,QUESTION 1: What nutritional/metabolic test(s)...,A 9-year-old boy presented with a 2-day histor...,metabolic disorders,DISEASE,associated with,low carnitine levels?Organic acidaemiasFatty a...,CHEMICAL
2,31019473,This is more evident during rapid growth (i.e....,Thiamine (vitamin B1) is an essential nutrient...,thiamine deficiency,NUTRITION,associated in,malnutrition,DISEASE
3,31019473,Thiamine deficiency contributes to a number of...,Thiamine (vitamin B1) is an essential nutrient...,confusion,DISEASE,reduce memory to,ataxia,DISEASE
4,31019473,Thiamine deficiency contributes to a number of...,Thiamine (vitamin B1) is an essential nutrient...,confusion,DISEASE,reduce memory to,death,DISEASE
5,31019473,Thiamine deficiency contributes to a number of...,Thiamine (vitamin B1) is an essential nutrient...,confusion,DISEASE,reduce memory to,muscle atrophy,DISEASE
6,31019473,Thiamine deficiency contributes to a number of...,Thiamine (vitamin B1) is an essential nutrient...,confusion,DISEASE,reduce memory to,severe encephalopathy,DISEASE
7,31019473,Thiamine deficiency contributes to a number of...,Thiamine (vitamin B1) is an essential nutrient...,confusion,DISEASE,reduce memory to,congestive heart failure,DISEASE
8,31019473,This review discusses the current knowledge on...,Thiamine (vitamin B1) is an essential nutrient...,thiamine supplementation,NUTRITION,have the putative beneficial effect in,autism spectrum disorder,MENTAL_HEALTH
9,21453474,"This is, to the best of our knowledge, the fir...",Autism is a complex neurodevelopmental disorde...,vitamin deficiency,BIOCHEMICAL,have a case of,autism spectrum disorder,MENTAL_HEALTH


In [10]:
def get_full_words(abbr:str, abstract:str, nlp)->str:
    """
    Get full words of a given abbreviation in sentence
    """
    doc = nlp(abstract)
    abbr_phrase = None
    for sent in doc.sents:
      find_abbr = re.compile(f"(.+) (\(|\[){abbr}(\)|\])").findall(sent.text)
      if len(find_abbr) > 0:
        abbr_phrase = find_abbr[0][0]
    if not abbr_phrase:
      return abbr

    abbr_phrase = nlp(abbr_phrase)
    full_words = []
    i = len(abbr_phrase) - 1
    while i >= 0:
      if abbr_phrase[i].pos_ not in ['ADJ', 'NOUN', 'PROPN', 'PART', 'NUM']:
        break
      full_words.append(abbr_phrase[i].text)
      i -= 1

    return ' '.join(full_words[::-1]).strip()

In [11]:
abbr_dict = {}
full_df = full_df.reset_index(drop=True)
print("Using get_full_words function:")
full_df['Full_E1'] = full_df['E1'].tolist()
full_df['Full_E2'] = full_df['E2'].tolist()

for idx, row in full_df.iterrows():
    abstract = row['Abstract']

    if idx%100 == 0:
      print(f"Row {idx}")

    # Full words of E1
    e1 = re.sub(r"total |further |other |many |much |several |some ", "", str(row['E1']))
    try:
      full_e1 = get_full_words(abbr=e1, abstract=abstract, nlp=gena_nlp)
      if e1 != full_e1:
        abbr_dict[e1] = full_e1
      full_df.loc[idx, "E1"] = e1
      full_df.loc[idx, "Full_E1"] = full_e1
    except:
      print(e1)


    # Full words of E2
    e2 = re.sub(r"total |further |other |many |much |several |some ", "", str(row['E2']))
    try:
      full_e2 = get_full_words(abbr=e2, abstract=abstract, nlp=gena_nlp)
      if e2 != full_e2:
        abbr_dict[e2] = full_e2
      full_df.loc[idx, "E2"] = e2
      full_df.loc[idx, "Full_E2"] = full_e2
    except:
      print(e2)

abbr_dict.items()

Using get_full_words function:
Row 0
Row 100
Row 200
Row 300
Row 400
Row 500
L-(3H glutamate
Row 600
Row 700
Row 800
Row 900
Row 1000
Row 1100
Row 1200
Row 1300
Row 1400
Row 1500
fluoxetine (±)-N-methyl-γ-[4-(trifluoromethyl)phenoxy]benzenepropanamine hydrochloride
fluoxetine (±)-N-methyl-γ-[4-(trifluoromethyl)phenoxy]benzenepropanamine hydrochloride
Row 1600
Row 1700
Row 1800
poly(lactic acid nanoparticles
poly(lactic acid nanoparticles
Row 1900
Row 2000
Row 2100
Histamine H(1 receptor antagonists
Row 2200
Row 2300
Row 2400
Row 2500
Row 2600
Row 2700
Row 2800
lactate clearance(LC
Row 2900
ATP MgSO(4
Row 3000
Row 3100
Row 3200
Row 3300
Row 3400
alpha 1-(14 %
Row 3500
effective 5-HT(1B/1D agonists
effective 5-HT(1B/1D agonists
Row 3600
Row 3700
Row 3800
Row 3900
Row 4000
Row 4100
vitamin B(6
Row 4200
Row 4300
Row 4400
Row 4500
Row 4600
Row 4700
Row 4800
B(12 deficiency
vitamin B(12
B(12 deficiency
vitamin B(12
Row 4900


dict_items([('TRD', 'Treatment-resistant depression'), ('NICCD', 'neonatal intrahepatic cholestasis'), ('CTLN2', 'adult-onset type II citrullinemia'), ('phenolic compounds', 'Alkylresorcinol'), ('ADP', 'Accidental dural puncture'), ('NLAAM', 'potent opiate N-desmethyl-l-alpha-noracetylmethadol'), ('VRD', 'NMDA-induced ventral root depolarization'), ('ADHD', 'Attention-deficit/hyperactivity disorder'), ('SRS', 'Snyder-Robinson Syndrome'), ('AA', 'asiatic acid'), ('PS', 'phosphatidylserine'), ('MDD', 'major depressive disorder'), ('long-acting dihydropyridine', 'nisoldipine'), ('nano-Se', ''), ('Nano-Se', ''), ('CRF', 'Corticotropin-Releasing Factor'), ('GABA', 'gamma-aminobutyric acid'), ('PPA', 'Propionic acid'), ('NAC', 'N-acetyl cysteine'), ('CUD', 'disorder'), ('5-HIAA', 'metabolite 5-hydroxyindoleacetic acid'), ('5-HT', 'cerebral serotonin'), ('TBI', 'traumatic brain injury'), ('PWS', 'Prader-Willi syndrome'), ('ABA', 'activity-based anorexia'), ('L-AG', 'GABA synthesis inhibitor L

In [ ]:
import periodictable
## Use dictionary and periodictable- for chemical realted terms
print("Using abbreviation dictionary:")
for idx, row in full_df.iterrows():
    if idx%100 == 0:
        print(f"Row {idx}")
    e1 = row['E1']
    e2 = row['E2']

    full_df.loc[idx, 'Full_E1'] = abbr_dict.get(e1, e1)
    full_df.loc[idx, 'Full_E2'] = abbr_dict.get(e2, e2)
    letters_1 = re.compile(r'([A-Za-z]+)').findall(e1)
    if len(letters_1) == 1:
      for el in periodictable.elements:
        if letters_1[0] == el.symbol:
            full_df.loc[idx, 'Full_E1'] = el.name

    letters_2 = re.compile(r'([A-Za-z]+)').findall(e2)
    if len(letters_2) == 1:
      for el in periodictable.elements:
        if letters_2[0] == el.symbol:
            full_df.loc[idx, 'Full_E2'] = el.name

Using abbreviation dictionary:
Row 0
Row 100
Row 200
Row 300
Row 400
Row 500
Row 600
Row 700
Row 800
Row 900
Row 1000
Row 1100
Row 1200
Row 1300
Row 1400
Row 1500
Row 1600
Row 1700
Row 1800
Row 1900
Row 2000
Row 2100
Row 2200
Row 2300
Row 2400
Row 2500
Row 2600
Row 2700
Row 2800
Row 2900
Row 3000
Row 3100
Row 3200
Row 3300
Row 3400
Row 3500
Row 3600
Row 3700
Row 3800
Row 3900
Row 4000
Row 4100
Row 4200
Row 4300
Row 4400
Row 4500
Row 4600
Row 4700
Row 4800
Row 4900


In [ ]:
print(len(full_df))
full_df = full_df.drop_duplicates(subset=['Full_E1', 'Relation', 'Full_E2']).reset_index(drop=True)
print(len(full_df))

4918
4670


# Get ID for all Entities from MeSH(Medical Subject Headings) database

In [ ]:
# Get new relations extracted
import pandas as pd
gena_normal_relation = full_df[["PMID", "Sentence", "E1", "Type_E1", "Relation", "E2", "Type_E2", "Full_E1", "Full_E2"]]
gena_normal_relation["ID_1"] = [""]*len(gena_normal_relation)
gena_normal_relation["ID_2"] = [""]*len(gena_normal_relation)
gena_normal_relation["Synonyms_1"] = [""]*len(gena_normal_relation)
gena_normal_relation["Synonyms_2"] = [""]*len(gena_normal_relation)
gena_normal_relation = gena_normal_relation[["PMID", "Sentence", "ID_1", "E1", "Type_E1", "Full_E1", "Synonyms_1", "Relation", "ID_2", "E2", "Type_E2", "Full_E2", "Synonyms_2"]]

In [ ]:
def find_mesh_info(term:str)->tuple:
    """
    Get MeSH info (ID, Title and Synonyms) of a given term
    """
    # The National Center for Biotechnology Information advances science and health by providing access to biomedical and genomic information.
    root_URL = "https://www.ncbi.nlm.nih.gov"
    URL = f"{root_URL}/mesh/?term=\"{term}\"+%5BText+Words%5D"
    results = BeautifulSoup(requests.get(URL).content, "html.parser")
    first_result = results.find("div", class_="rslt")
    if first_result:
        detailed_URL = root_URL + first_result.find("a")['href']
        results = BeautifulSoup(requests.get(detailed_URL).content, "html.parser")
    elif results.find("span", class_="icon", string="No items found."):
        return ("No_ID_MeSH", term, "No synonyms")

    title = results.find("h1", class_="title").text
    mesh_id = results.find("p", text=re.compile(r'MeSH Unique ID.*')).text.replace("MeSH Unique ID: ", "")
    results_ab = results.find("div", class_="rprt abstract")
    mesh_synonyms = results_ab.findChildren("ul", recursive=False)
    if len(mesh_synonyms) > 0:
        all_synonyms = [syn.text for syn in BeautifulSoup(str(mesh_synonyms[0]), "html.parser").find_all("li")]
        return (mesh_id, title, '|'.join(all_synonyms))
    else:
        return (mesh_id, title, "No synonyms")

In [ ]:
term ="meropenem"
term_lemma = ' '.join([token.lemma_ for token in gena_nlp(term)])
find_mesh_info(term_lemma)

<ipython-input-18-df8d4a349909>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  mesh_id = results.find("p", text=re.compile(r'MeSH Unique ID.*')).text.replace("MeSH Unique ID: ", "")


('D000077731',
 'Meropenem',
 '3-(5-Dimethylcarbamoylpyrrolidin-3-ylthio)-6-(1-hydroxyethyl)-4-methyl-7-oxo-1-azabicyclo(3.2.0)hept-2-ene-2-carboxylic acid|Merrem|Ronem|Penem|SM 7338|SM-7338|SM7338')

In [ ]:
from time import sleep
gena_normal_relation['MeSH_E1'] = ['']*len(gena_normal_relation)
gena_normal_relation['MeSH_E2'] = ['']*len(gena_normal_relation)

mesh_dict = {}

for idx, row in gena_normal_relation.iterrows():
    print(idx)
    if idx < 7759:
      continue
    if not mesh_dict.get(str(row['Full_E1']), False):
      try:
        mesh_results_1 = find_mesh_info(str(row['Full_E1']))
        sleep(2)
      except:
        sleep(120)
        mesh_results_1 = find_mesh_info(str(row['Full_E1']))
        sleep(2)
      mesh_dict[str(row['Full_E1'])] = mesh_results_1

    if not mesh_dict.get(str(row['Full_E2']), False):
      try:
        mesh_results_2 = find_mesh_info(str(row['Full_E2']))
        sleep(2)
      except:
        sleep(120)
        mesh_results_2 = find_mesh_info(str(row['Full_E2']))
        sleep(2)
      mesh_dict[str(row['Full_E2'])] = mesh_results_2

    # Fill into the dataframe
    # Entities 1
    gena_normal_relation.loc[idx, 'ID_1'] = mesh_dict[str(row['Full_E1'])][0]
    gena_normal_relation.loc[idx, 'MeSH_E1'] = mesh_dict[str(row['Full_E1'])][1]
    gena_normal_relation.loc[idx, 'Synonyms_1'] = mesh_dict[str(row['Full_E1'])][2]

    # Entities 2
    gena_normal_relation.loc[idx, 'ID_2'] = mesh_dict[str(row['Full_E2'])][0]
    gena_normal_relation.loc[idx, 'MeSH_E2'] = mesh_dict[str(row['Full_E2'])][1]
    gena_normal_relation.loc[idx, 'Synonyms_2'] = mesh_dict[str(row['Full_E2'])][2]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Get ID from Ontologies

In [ ]:
GENERAL_PATH = "/content/drive/MyDrive/gena-db-master/src/DATASET"
DATA_PATH = f"{GENERAL_PATH}/data"
RESULT_PATH = f"{GENERAL_PATH}/results"

In [ ]:
import pandas as pd
import numpy as np

# Define the path to the DOID.csv file
DOID_PATH = "/content/drive/MyDrive/gena-db-master/src/DATASET/data/resource_csv/DOID.csv"

# Define the function to concatenate DataFrames
def concat_df(file_name: str, all_ont_df: pd.DataFrame) -> pd.DataFrame:
    """
    Concatenate a new DataFrame from a file to the given DataFrame.
    """
    if file_name.endswith('CHEBI.csv'):
        return all_ont_df
    else:
        ont_df = pd.read_csv(file_name)[['Class ID', 'Preferred Label', 'Synonyms']]
        if file_name == "APADISORDERS.csv":
            ont_df['Preferred Label'] = ont_df['Preferred Label'].map(lambda x: x.replace("_", " ").lower())
        else:
            ont_df['Preferred Label'] = ont_df['Preferred Label'].map(lambda x: x.lower() if isinstance(x, str) else "")
        return pd.concat([all_ont_df, ont_df])

# list of ontology files
ontologies = ['SYMP.csv', 'DOID.csv', 'CHEBI.csv', 'APADISORDERS.csv', 'ASDPTO.csv', 'FMA.csv', 'MFOMD.csv']

full_ont_df = pd.read_csv(f'{DATA_PATH}/resource_csv/{ontologies[0]}')[['Class ID', 'Preferred Label', 'Synonyms']]
full_ont_df['Preferred Label'] = full_ont_df['Preferred Label'].map(lambda x: x.lower())

# Concatenate data from other ontology files
for ont in ontologies[1:]:
    full_ont_df = concat_df(f'{DATA_PATH}/resource_csv/{ont}', full_ont_df)

full_ont_df.drop_duplicates(subset=['Class ID'], inplace=True, ignore_index=True)

full_ont_df["Vector"] = full_ont_df['Preferred Label'].map(lambda x: np.array(gena_nlp(x).vector))

# Display the first 5 rows of the DataFrame
full_ont_df.head(5)


<ipython-input-25-4a46898801bc>:15: DtypeWarning: Columns (8,51,65,71,75,79) have mixed types. Specify dtype option on import or set low_memory=False.
  ont_df = pd.read_csv(file_name)[['Class ID', 'Preferred Label', 'Synonyms']]
<ipython-input-25-4a46898801bc>:15: DtypeWarning: Columns (41,71,75,77,99,100,103,113,114,138,140,156,157,158,178,198) have mixed types. Specify dtype option on import or set low_memory=False.
  ont_df = pd.read_csv(file_name)[['Class ID', 'Preferred Label', 'Synonyms']]


,Class ID,Preferred Label,Synonyms,Vector
0,http://purl.obolibrary.org/obo/SYMP_0000149,sudden onset of severe chills,NaN,"[-0.07547201, 0.31105042, -0.02869026, -0.1448..."
1,http://purl.obolibrary.org/obo/SYMP_0000259,dry hacking cough,NaN,"[-0.08133, 0.46636868, -0.01648433, -0.0966516..."
2,http://purl.obolibrary.org/obo/SYMP_0000729,pulmonary consolidation,NaN,"[-0.022594575, 0.311132, 0.176519, -0.0930199,..."
3,http://purl.obolibrary.org/obo/SYMP_0000746,enanthem,NaN,"[0.397117, 0.552231, -0.245164, -0.408734, -0...."
4,http://purl.obolibrary.org/obo/SYMP_0000412,anxiety,NaN,"[-0.568638, -0.0604911, -0.0904757, -0.11993, ..."


In [ ]:
# Protein Dictionary
protein_df = pd.read_csv(f"{DATA_PATH}/resource_csv/PR.csv")
protein_dict = {}
for idx, row in protein_df.iterrows():
  if isinstance(row['Preferred Label'], str):
    synonyms = [syn.lower() for syn in (row["Synonyms"].split("|") if isinstance(row["Synonyms"], str) else [])]
    protein_dict[row['Class ID']] = [row['Preferred Label'].lower()] + synonyms
len(list(protein_dict.keys()))

In [ ]:
def find_protein(term:str, protein_dict:dict)->tuple:
  """
  Find protein name of a term if possible
  """
  for key in protein_dict.keys():
    for protein in protein_dict[key][1:]:
      if term in re.sub('[^A-Za-z0-9]+', ' ', protein).split():
        return (key, protein_dict[key][0], '|'.join(protein_dict[key][1:]) if len(protein_dict[key]) > 1 else "No Synonyms")
  return ("No_ID_MeSH", term, "No synonyms")

def cosine_similar(vec_1:ndarray, vec_2:ndarray):
  """
  Cosine distance of 2 vectors in SpaCy
  """
  return (vec_1.dot(vec_2))/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))

def most_similar(ont_df:DataFrame, term:str, protein_dict:dict, nlp)->tuple:
  """
  """
  term_vector = np.array(nlp(term).vector)
  max_result = ont_df["Vector"].map(lambda x: cosine_similar(term_vector, x))
  max_similar = max_result.max()
  max_index = max_result.argmax()

  if max_similar > 0.95:
    synonyms = ont_df.loc[max_index, 'Synonyms'] if isinstance(ont_df.loc[max_index, 'Synonyms'], str) else "No synonyms"
    return (ont_df.loc[max_index, 'Class ID'], ont_df.loc[max_index, 'Preferred Label'], synonyms)
  else:
    return find_protein(term, protein_dict)

In [ ]:
ontology_dict = {}
relation_df = gena_normal_relation[["PMID", "Sentence", "ID_1", "E1", "Type_E1", "Full_E1", "MeSH_E1", "Synonyms_1", "Relation", "ID_2", "E2", "Type_E2", "Full_E2", "MeSH_E2", "Synonyms_2"]]
for idx, row in relation_df.iterrows():
  print(idx)

  if (row["ID_1"] != "No_ID_MeSH" and row["ID_2"] != "No_ID_MeSH"):
    continue

  if row["ID_1"] == "No_ID_MeSH":
    if not ontology_dict.get(row['E1'], False):
      term_1 = ' '.join(word.lemma_ for word in gena_nlp(row["Full_E1"]))
      result_1 = most_similar(full_ont_df, term_1, protein_dict, gena_nlp)
      ontology_dict[row['E1']] = result_1
    relation_df.loc[idx, 'ID_1'], relation_df.loc[idx, 'MeSH_E1'], relation_df.loc[idx, 'Synonyms_1'] = ontology_dict[row['E1']]

  if row["ID_2"] == "No_ID_MeSH":
    if not ontology_dict.get(row['E2'], False):
      term_2 = ' '.join(word.lemma_ for word in gena_nlp(row["Full_E2"]))
      result_2 = most_similar(full_ont_df, term_2, protein_dict, gena_nlp)
      ontology_dict[row['E2']] = result_2
    relation_df.loc[idx, 'ID_2'], relation_df.loc[idx, 'MeSH_E2'], relation_df.loc[idx, 'Synonyms_2'] = ontology_dict[row['E2']]


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
print(len(relation_df[relation_df['ID_1'] != "No_ID_MeSH"]), len(relation_df[relation_df['ID_2'] != "No_ID_MeSH"]))
relation_df.to_csv(f"{RESULT_PATH}/raw_result/gena_update_Ontologies_full_3.csv", index=False)


4670 4670


# Create new ID for new terms

In [ ]:
relation_df = pd.concat([pd.read_csv(f"{RESULT_PATH}/raw_result/gena_update_Ontologies_full.csv"),
                         pd.read_csv(f"{RESULT_PATH}/raw_result/gena_update_Ontologies_full_2.csv"),
                         pd.read_csv(f"{RESULT_PATH}/raw_result/gena_update_Ontologies_full_3.csv")]).reset_index(drop=True)
relation_df.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
1,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
2,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
3,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
4,27584019,Obesity is associated with reduced testosteron...,D063766,Obesity,DISEASE,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",be associated with,No_ID_MeSH,reduced testosterone,CHEMICAL,reduced testosterone,reduced testosterone,No synonyms


In [ ]:
available_df = pd.read_csv(f"{RESULT_PATH}/final_data/available_fad.csv")
available_df.columns = ['ID_1', 'E1', 'Relation', 'ID_2', 'E2']
available_df['Full_E1'], available_df['Full_E2'] = available_df['E1'].tolist(), available_df['E2'].tolist()
available_df['MeSH_E1'], available_df['MeSH_E2'] = available_df['E1'].tolist(), available_df['E2'].tolist()
available_df['Type_E1'] = ['CHEMICAL']*2434 + ['DISEASE']*(len(available_df)-2434)
available_df['Type_E2'] = ['DISEASE']*2434 + ['NUTRITION']*(len(available_df)-2434)

In [ ]:
available_df['PMID'] = ['BC5CDR']*2434 + ['DOID']*(len(available_df)-2434)
available_df['Sentence'] = available_df['Synonyms_1'] = available_df['Synonyms_2'] = ['']*len(available_df)
available_df = available_df[relation_df.columns]
relation_df = pd.concat([relation_df, available_df], axis=0).dropna().reset_index(drop=True)
relation_df.head(5)

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2
0,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
1,31196764,"Flibanserin, a multifunctional serotonin recep...",C098107,Flibanserin,CHEMICAL,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
2,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,No_ID_MeSH,acquired generalized hypoactive sexual desire ...,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms
3,31196764,"Flibanserin, a multifunctional serotonin recep...",No_ID_MeSH,multifunctional serotonin receptor agonist,CHEMICAL,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,be approved in united states and canada for tr...,D020018,HSDD,DISEASE,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions..."
4,27584019,Obesity is associated with reduced testosteron...,D063766,Obesity,DISEASE,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",be associated with,No_ID_MeSH,reduced testosterone,CHEMICAL,reduced testosterone,reduced testosterone,No synonyms


In [ ]:
relation_df[relation_df['ID_1'].isna()]

,PMID,Sentence,ID_1,E1,Type_E1,Full_E1,MeSH_E1,Synonyms_1,Relation,ID_2,E2,Type_E2,Full_E2,MeSH_E2,Synonyms_2


In [ ]:
new_count = 0
new_term_id_dict = {}
for idx, row in relation_df.iterrows():
  print(idx)
  if row['ID_1'].startswith("No"):
    if new_term_id_dict.get(row['E1']):
      relation_df.loc[idx, 'ID_1'] = new_term_id_dict[row['E1']]
    else:
      new_id = "NEW" + "{:05d}".format(new_count)
      relation_df.loc[idx, 'ID_1'] = new_id
      new_term_id_dict[row['E1']] = new_id
      new_count += 1

  if row['ID_2'].startswith("No"):
    if new_term_id_dict.get(row['E2']):
      relation_df.loc[idx, 'ID_2'] = new_term_id_dict[row['E2']]
    else:
      new_id = "NEW" + "{:05d}".format(new_count)
      relation_df.loc[idx, 'ID_2'] = new_id
      new_term_id_dict[row['E2']] = new_id
      new_count += 1
# Check if there is no-id terms
len(relation_df[relation_df['ID_1'] == 'No_ID_MeSH']), len(relation_df[relation_df['ID_2'] == 'No_ID_MeSH'])

Streaming output truncated to the last 5000 lines.
38433
38434
38435
38436
38437
38438
38439
38440
38441
38442
38443
38444
38445
38446
38447
38448
38449
38450
38451
38452
38453
38454
38455
38456
38457
38458
38459
38460
38461
38462
38463
38464
38465
38466
38467
38468
38469
38470
38471
38472
38473
38474
38475
38476
38477
38478
38479
38480
38481
38482
38483
38484
38485
38486
38487
38488
38489
38490
38491
38492
38493
38494
38495
38496
38497
38498
38499
38500
38501
38502
38503
38504
38505
38506
38507
38508
38509
38510
38511
38512
38513
38514
38515
38516
38517
38518
38519
38520
38521
38522
38523
38524
38525
38526
38527
38528
38529
38530
38531
38532
38533
38534
38535
38536
38537
38538
38539
38540
38541
38542
38543
38544
38545
38546
38547
38548
38549
38550
38551
38552
38553
38554
38555
38556
38557
38558
38559
38560
38561
38562
38563
38564
38565
38566
38567
38568
38569
38570
38571
38572
38573
38574
38575
38576
38577
38578
38579
38580
38581
38582
38583
38584
38585
38586
38587
38588
38589
38590
3

(0, 0)

# Prepare data for Knowledge Graph

## ID and terms from papers corresponding

In [ ]:
term_df = relation_df[['ID_1', 'E1', 'Full_E1', 'MeSH_E1', 'Synonyms_1', 'Type_E1', 'ID_2', 'E2', 'Full_E2', 'MeSH_E2', 'Synonyms_2', 'Type_E2']].fillna({'Synonyms_1': 'No synonyms', 'Synonyms_2': 'No synonyms'})
term_df.columns = ['ID_1', 'Paper_term_1', 'Full_paper_term_1', 'Standard_term_1', 'Synonyms_1', 'Type_E1', 'ID_2', 'Paper_term_2', 'Full_paper_term_2', 'Standard_term_2', 'Synonyms_2', 'Type_E2']
term_df.head(5)

,ID_1,Paper_term_1,Full_paper_term_1,Standard_term_1,Synonyms_1,Type_E1,ID_2,Paper_term_2,Full_paper_term_2,Standard_term_2,Synonyms_2,Type_E2
0,C098107,Flibanserin,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,CHEMICAL,NEW00000,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms,MENTAL_HEALTH
1,C098107,Flibanserin,Flibanserin,flibanserin [Supplementary Concept],Benzimidazoles,CHEMICAL,D020018,HSDD,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions...",DISEASE
2,NEW00001,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,CHEMICAL,NEW00000,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,acquired generalized hypoactive sexual desire ...,No synonyms,MENTAL_HEALTH
3,NEW00001,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,multifunctional serotonin receptor agonist,No synonyms,CHEMICAL,D020018,HSDD,hypoactive sexual desire disorder,"Sexual Dysfunctions, Psychological","Dysfunction, Psychological Sexual|Dysfunctions...",DISEASE
4,D063766,Obesity,Obesity,Pediatric Obesity,"Obesity, Pediatric|Obesity in Childhood|Childh...",DISEASE,NEW00002,reduced testosterone,reduced testosterone,reduced testosterone,No synonyms,CHEMICAL


In [ ]:
unique_id = {}
for idx, row in term_df.iterrows():
    for k in range(1, 3):
        unique_id[row[f'ID_{k}']] = unique_id.get(row[f'ID_{k}'], {'preferred_terms': row[f'Standard_term_{k}'], 'label': row[f'Type_E{k}'], 'paper_terms': set(), 'synonyms': set()})
        if row[f'Paper_term_{k}'] != '':
          unique_id[row[f'ID_{k}']]['paper_terms'].add(row[f'Paper_term_{k}'])
        if row[f'Full_paper_term_{k}'] != '':
          unique_id[row[f'ID_{k}']]['paper_terms'].add(row[f'Full_paper_term_{k}'])
        if row[f'Synonyms_{k}'] != 'No synonyms' or row[f'Synonyms_{k}'] != '':
          unique_id[row[f'ID_{k}']]['synonyms'].update(row[f'Synonyms_{k}'].split('|'))

In [ ]:
terms_tuple = []
for id, terms in unique_id.items():
    terms_tuple.append((id, terms['label'], terms['preferred_terms'], '|'.join(list(terms['paper_terms'].union(terms['synonyms'])))))
ID_terms_df = pd.DataFrame(terms_tuple, columns=['ID', 'Label', 'Preferred label', 'Synonyms'])
ID_terms_df.head(5)

,ID,Label,Preferred label,Synonyms
0,C098107,CHEMICAL,flibanserin [Supplementary Concept],flibanserin|Flibanserin|Benzimidazoles
1,NEW00000,MENTAL_HEALTH,acquired generalized hypoactive sexual desire ...,No synonyms|acquired generalized hypoactive se...
2,D020018,DISEASE,"Sexual Dysfunctions, Psychological","|Dysfunction, Psychosexual|frigidity|Orgasmic ..."
3,NEW00001,CHEMICAL,multifunctional serotonin receptor agonist,No synonyms|multifunctional serotonin receptor...
4,D063766,DISEASE,Pediatric Obesity,"Overweight, Childhood|Obesity in Childhood|Obe..."


In [ ]:
# Check if each entity has unique ID
len(ID_terms_df), len(ID_terms_df['ID'].unique())

(27008, 27008)

In [ ]:
ID_terms_df.to_csv(f"{RESULT_PATH}/final_data/ID_to_ENTITIES_final.csv", index=False)

## < ID_1, Relation, ID_2 > tuples

In [ ]:
id_relation_df = relation_df.dropna().drop_duplicates(subset=['ID_1', 'Relation', 'ID_2']).reset_index(drop=True)
id_relation_df.to_csv(f"{RESULT_PATH}/final_data/gena_data_final.csv", index=False)

# Final Results

In [ ]:
import pandas as pd

In [ ]:
id_relation_df = pd.read_csv(f"{RESULT_PATH}/final_data/gena_data_final.csv")
available_data = pd.read_csv(f"{RESULT_PATH}/final_data/available_fad.csv")

## 1. Numbers of unique relations

In [ ]:
# All relations
print("Total relations:", len(id_relation_df))

Total relations: 42494


In [ ]:
n_from_old_relations = len(available_data)

# New relations
print("Detected relations:", len(id_relation_df) - n_from_old_relations)

# Old relations
print("Available relations:", n_from_old_relations)

Detected relations: 39932
Available relations: 2562


## 2. Numbers of new relation from Old Entities

In [ ]:
old_relations = id_relation_df.iloc[-n_from_old_relations:, :]
old_couple_dict = {(id_1, id_2): [relation] for (id_1, id_2, relation) in zip(old_relations['ID_1'], old_relations['ID_2'], old_relations['Relation'])}

new_relations = id_relation_df.iloc[:-n_from_old_relations, :]
for (id_1, id_2, relation) in zip(new_relations['ID_1'], new_relations['ID_2'], new_relations['Relation']):
  if (id_1, id_2) in old_couple_dict.keys():
    old_couple_dict[(id_1, id_2)].append(relation)

new_relation_from_old_tuples = []
for couple in old_couple_dict.keys():
  if len(old_couple_dict[couple]) > 1:
    for relation in old_couple_dict[couple][1:]:
      new_relation_from_old_tuples.append((couple[0], couple[1], old_couple_dict[couple][0], relation))

new_relation_from_old_df = pd.DataFrame(new_relation_from_old_tuples, columns=['ID_1', 'ID_2', 'Old_Relation', 'New_Relation'])
print(f"There are {len(new_relation_from_old_df)} relations for old couple-entities.")
new_relation_from_old_df.head(len(new_relation_from_old_df))

There are 23 relations for old couple-entities.


,ID_1,ID_2,Old_Relation,New_Relation
0,D003024,D000380,CID,cause fatal hematopoietic toxicity as
1,D017367,D001714,CID,be prescribed to patients with
2,D018021,D011141,CID,include
3,D003024,D009205,CID,be effective medication for
4,D004837,D006323,CID,have the use to
5,D010433,D012640,CID,induce
6,D007649,D006212,CID,cause
7,D007649,D006212,CID,be complicated by
8,D007649,D006212,CID,induce
9,D002110,D012640,CID,have the influence on


## 3. Numbers of totally new relations

In [ ]:
totally_relations = len(new_relations) - len(new_relation_from_old_df)
print("Totally new relations:", totally_relations)

Totally new relations: 39909


## 4. Numbers of New Entities

In [ ]:
print("Total numbers of entities:", len(ID_terms_df))

Total numbers of entities: 27008


In [ ]:
new_id = len(ID_terms_df[ID_terms_df['ID'].str.startswith('NEW')])
print("Numbers of new entities:", new_id)

Numbers of new entities: 20257
